<a href="https://colab.research.google.com/github/kankerllijer/OOP/blob/main/Proyecto_cumple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install -q streamlit python-dateutil
!npm install localtunnel
!pip install --upgrade streamlit streamlit-calendar
!pip install streamlit email-validator python-dateutil schedule pytz
!pip install streamlit python-dateutil schedule pytz

!pip install -r requirements.txt


⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 1s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦Requirement already satisfied: streamlit in /usr/local/lib/python3.11/dist-packages (1.45.1)
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [90]:
%%writefile __init__.py

"""Paquete para gestión de cumpleaños.

Módulos:
    contacto: Define la estructura de datos para contactos.
    gestor_cumpleaños: Lógica de negocio para manejar fechas y almacenamiento.
    enviar_correos: Funcionalidad de envío de correos electrónicos.
"""

from .contacto import Contacto
from .gestor_cumpleaños import GestorCumpleaños
from .enviar_correos import EnviarCorreos

__all__ = ["Contacto", "GestorCumpleaños", "EnviarCorreos"]

Writing __init__.py


In [91]:
%%writefile contacto.py
from datetime import date

class Contacto:
    """Representa un contacto con información personal relevante para cumpleaños.

    Atributos:
        nombre (str): Nombre completo del contacto.
        fecha_nacimiento (date): Fecha de nacimiento en formato YYYY-MM-DD.
        correo (str): Dirección de correo electrónico del contacto.
    """

    def __init__(self, nombre: str, fecha_nacimiento: date, correo: str):
        """Inicializa un nuevo contacto.

        Args:
            nombre (str): Nombre del contacto. No puede estar vacío.
            fecha_nacimiento (date): Fecha de nacimiento válida.
            correo (str): Correo electrónico válido.
        """
        self.nombre = nombre
        self.fecha_nacimiento = fecha_nacimiento
        self.correo = correo

Writing contacto.py


In [92]:
%%writefile gestor_cumpleaños.py

from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from typing import List, Tuple
from .contacto import Contacto
from .almacenamiento import Almacenamiento

class GestorCumpleaños:

    """Maneja el almacenamiento y cálculo de fechas de cumpleaños.

    Responsable de:
    - Cargar/guardar contactos desde/hacia CSV
    - Calcular próximos cumpleaños
    - Ordenar eventos por proximidad
    """

    def __init__(self, almacenamiento: Almacenamiento):
        self.almacenamiento = almacenamiento
        self.contactos = self.almacenamiento.cargar_contactos()

    def agregar_contacto(self, contacto: Contacto):
        self.contactos.append(contacto)
        self.almacenamiento.guardar_contactos(self.contactos)

    def obtener_proximo_cumpleaños(self, contacto: Contacto) -> date:

        """Calcula la próxima fecha de cumpleaños para un contacto.
        Args:
            contacto (Contacto): Contacto para calcular su cumpleaños.
        Returns:
            date: Próxima fecha de cumpleaños ajustada al año actual.
        Nota:
            Maneja casos especiales como 29 de febrero, ajustando a 28/02 en años no bisiestos.
        """

        hoy = date.today()
        año_actual = hoy.year

        try:
            proximo_cumple = contacto.fecha_nacimiento.replace(year=año_actual)

            # Manejo especial para 29 de febrero
            if proximo_cumple.month == 2 and proximo_cumple.day == 29:
                # Ajustar a 28/02 si el año actual no es bisiesto
                if not self._es_año_bisiesto(año_actual):
                    proximo_cumple = date(año_actual, 3, 1)  # Mover a 1 de marzo
        except ValueError:
            proximo_cumple = date(año_actual, 3, 1)

        if proximo_cumple < hoy:
            proximo_cumple = proximo_cumple.replace(year=año_actual + 1)
            if proximo_cumple.month == 2 and proximo_cumple.day == 29:
                if not self._es_año_bisiesto(año_actual + 1):
                    proximo_cumple = date(año_actual + 1, 3, 1)

        return proximo_cumple

    def _es_año_bisiesto(self, año: int) -> bool:
        """Determina si un año es bisiesto según el calendario gregoriano.

        Args:
            año: Año a validar (entero positivo).

        Returns:
            bool: True si es bisiesto, False en caso contrario.
        """

        return año % 4 == 0 and (año % 100 != 0 or año % 400 == 0)


    def obtener_proximos_cumpleaños(self) -> List[Tuple[Contacto, int]]:

        """Genera lista de cumpleaños ordenados por proximidad.

        Returns:
            List[Tuple[Contacto, int]]: Lista de tuplas (Contacto, días restantes),
            ordenada ascendentemente por días.
        """

        hoy = date.today()
        proximos = []
        for contacto in self.contactos:
            proximo_cumple = self.obtener_proximo_cumpleaños(contacto)
            dias_restantes = (proximo_cumple - hoy).days
            if dias_restantes < 0:  # Nunca debería ocurrir, pero por seguridad
                proximo_cumple = proximo_cumple.replace(year=hoy.year + 1)
                dias_restantes = (proximo_cumple - hoy).days
            proximos.append((contacto, dias_restantes))
        return sorted(proximos, key=lambda x: x[1])

    # En gestor_cumpleaños.py
    def es_cumpleaños_hoy(self, contacto: Contacto) -> bool:
            proximo = self.obtener_proximo_cumpleaños(contacto)
            return proximo == date.today()



Writing gestor_cumpleaños.py


In [93]:
%%writefile enviar_correos.py

import smtplib
from email.mime.text import MIMEText

class EnviarCorreos:
    """Maneja el envío de correos electrónicos mediante SMTP seguro (STARTTLS)."""

    def __init__(self, servidor: str, puerto: int, remitente: str, contraseña: str):

        """Configura conexión SMTP.

        Args:
            servidor (str): Dirección del servidor SMTP (ej: smtp.gmail.com).
            puerto (int): Puerto SMTP (ej: 587 para TLS).
            remitente (str): Correo del remitente.
            contraseña (str): Contraseña de aplicación o cuenta.
        """

        self.servidor = servidor
        self.puerto = puerto
        self.remitente = remitente
        self.contraseña = contraseña

    def enviar_correo(self, destino: str, asunto: str, mensaje: str) -> bool:

        """Envía un correo electrónico.

        Args:
            destino (str): Correo del destinatario.
            asunto (str): Línea de asunto.
            mensaje (str): Cuerpo del mensaje (formato texto plano).

        Returns:
            bool: True si el envío fue exitoso, False en caso contrario.

        Raises:
            SMTPException: Si ocurre un error durante el envío.
        """
        try:
            # Crear mensaje MIME
            msg = MIMEText(mensaje)
            msg["Subject"] = asunto
            msg["From"] = self.remitente
            msg["To"] = destino

            # Conexión SMTP con STARTTLS
            with smtplib.SMTP(self.servidor, self.puerto) as server:
                server.starttls()
                server.login(self.remitente, self.contraseña)
                server.sendmail(self.remitente, destino, msg.as_string())
                return True
        except Exception as e:
            print(f"Error al enviar correo: {str(e)}")
            return False


Writing enviar_correos.py


In [94]:
%%writefile almacenamiento.py

import csv
from datetime import datetime
from typing import List
from .contacto import Contacto
from abc import ABC, abstractmethod

class Almacenamiento(ABC):
    """Interfaz para estrategias de almacenamiento."""

    @abstractmethod
    def cargar_contactos(self) -> List[Contacto]:

        """Carga todos los contactos desde el almacenamiento.

        Returns:
            List[Contacto]: Lista de contactos recuperados.
            Lista vacía si no hay datos o hay error.
        """

        pass

    @abstractmethod
    def guardar_contactos(self, contactos: List[Contacto]):

        """Persiste la lista completa de contactos.

        Args:
            contactos: Lista de objetos Contacto a guardar.
        """

        pass

class AlmacenamientoCSV(Almacenamiento):
    """Implementación concreta de almacenamiento en archivo CSV.

    Args:
        archivo: Ruta del archivo CSV. Por defecto: 'contactos.csv'.
    """

    def __init__(self, archivo: str = "contactos.csv"):
        self.archivo = archivo

    def cargar_contactos(self) -> List[Contacto]:
        try:
            with open(self.archivo, 'r') as f:
                return [
                    Contacto(
                        fila['nombre'],
                        datetime.strptime(fila['fecha_nacimiento'], '%Y-%m-%d').date(),
                        fila['correo']
                    ) for fila in csv.DictReader(f)
                ]
        except FileNotFoundError:
            return []

    def guardar_contactos(self, contactos: List[Contacto]):
        with open(self.archivo, 'w', newline='') as f:
            escritor = csv.DictWriter(f, fieldnames=['nombre', 'fecha_nacimiento', 'correo'])
            escritor.writeheader()
            for contacto in contactos:
                escritor.writerow({
                    'nombre': contacto.nombre,
                    'fecha_nacimiento': contacto.fecha_nacimiento.strftime('%Y-%m-%d'),
                    'correo': contacto.correo
                })

Overwriting almacenamiento.py


In [95]:
%%writefile main.py

import streamlit as st
import os
import sys
import csv
import random
import smtplib
import threading
import pytz

from datetime import datetime
from datetime import date
sys.path.append(os.getcwd())
from streamlit_calendar import calendar
from gestor_cumpleaños.gestor_cumpleaños import GestorCumpleaños
from gestor_cumpleaños.contacto import Contacto
from gestor_cumpleaños.enviar_correos import EnviarCorreos
from gestor_cumpleaños.almacenamiento import AlmacenamientoCSV
from email_validator import validate_email, EmailNotValidError

def main():
    st.set_page_config(
        page_title="Gestor de Cumpleaños",
        page_icon="🎂",
        layout="wide"
    )

    # Configuración SMTP
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_user = ""
    smtp_pass = ""

    if os.path.exists("smtp_config.csv"):
        with open("smtp_config.csv", newline="") as f:
            server, port, user, pwd = next(csv.reader(f))
            smtp_server = server
            smtp_port = int(port)
            smtp_user = user
            smtp_pass = pwd

    # Inicializar gestor
    almacenamiento = AlmacenamientoCSV()
    gestor = GestorCumpleaños(almacenamiento)

    # --- Pestañas ---
    tab1, tab2 = st.tabs([
        "📝 Agregar Contactos / 📅 Próximos Cumpleaños",
        "📧 Gestión de Correos"
    ])

    # --- Pestaña 1 ---
    with tab1:
        col1, col2 = st.columns([2, 3])

        # Columna 1: Formulario y lista
        with col1:
            with st.form("nuevo_contacto", clear_on_submit=True):
                st.subheader("➕ Nuevo Contacto")
                nombre = st.text_input("Nombre*")

                try:
                    fecha_nac = st.date_input("Fecha de nacimiento*", min_value=date(1900, 1, 1))
                    if fecha_nac > date.today():
                        st.error("⚠️ La fecha de nacimiento no puede ser futura.")
                except ValueError as e:
                    st.error(f"Error en fecha: {str(e)}")

                correo = st.text_input("Correo*")

                if st.form_submit_button("💾 Guardar Contacto"):
                    if not nombre or not correo:
                        st.error("⚠️ Campos obligatorios faltantes")
                    else:
                        try:
                            valid = validate_email(correo)
                            correo = valid.email
                            gestor.agregar_contacto(Contacto(nombre, fecha_nac, correo))
                            gestor.almacenamiento.guardar_contactos(gestor.contactos)
                            st.rerun()
                        except EmailNotValidError as e:
                            st.error(f"⚠️ Correo inválido: {str(e)}")

            st.subheader("🎉 Próximos Cumpleaños")
            if not gestor.contactos:
                st.warning("No hay contactos registrados")
            else:
                for contacto, dias in gestor.obtener_proximos_cumpleaños():
                    st.markdown(f"""
                    **{contacto.nombre}**
                    🗓️ {contacto.fecha_nacimiento.strftime('%d/%m')}
                    📧 {contacto.correo}
                    ⏳ **{dias}** días restantes
                    """)

        # Columna 2: Calendario
        with col2:
            st.subheader("📆 Calendario de Eventos")
            eventos = [
                {
                    "title": f"🎂 {c.nombre}",
                    "start": date.today().replace(
                        month=c.fecha_nacimiento.month,
                        day=c.fecha_nacimiento.day
                    ).isoformat(),
                    "color": "#FF6B6B"
                } for c in gestor.contactos
            ]
            calendar(
                events=eventos,
                options={
                    "initialView": "dayGridMonth",
                    "locale": "es",
                    "headerToolbar": {
                        "left": "prev,next today",
                        "center": "title",
                        "right": "dayGridMonth,dayGridWeek"
                    }
                }
            )

    # --- Pestaña 2 ---
    with tab2:
        config_col, msg_col = st.columns([2, 3])

        with config_col:
            with st.expander("⚙️ Configuración SMTP", expanded=True):
                with st.form("smtp_config"):
                    smtp_server_input = st.text_input("Servidor", smtp_server)
                    smtp_port_input = st.number_input("Puerto", value=smtp_port)
                    smtp_user_input = st.text_input("Correo", smtp_user)
                    smtp_pass_input = st.text_input("Contraseña", type="password", value=smtp_pass)

                    if st.form_submit_button("💾 Guardar"):
                        with open("smtp_config.csv", "w", newline="") as f:
                            f.write(f"{smtp_server_input},{smtp_port_input},{smtp_user_input},{smtp_pass_input}")
                        st.success("¡Configuración guardada!")
                        st.rerun()

            fecha_prueba = st.date_input(
                "🔧 Simular fecha",
                value=datetime.now(pytz.timezone("America/Mexico_City")).date()
            )

            st.subheader("👥 Destinatarios")
            options = [f"{c.nombre} <{c.correo}>" for c in gestor.contactos]
            selected = st.multiselect("Seleccionar:", options=options, default=options)
            correos_destino = [s.split("<")[1].replace(">", "") for s in selected]

            st.subheader("🔠 Tipo de Mensaje")
            tipo_mensaje = st.radio(
                "Modo:",
                options=["🎲 Aleatorio", "✉️ Personalizado"],
                horizontal=True
            )

            mensaje_a_enviar = ""
            if tipo_mensaje == "🎲 Aleatorio":
                try:
                    with open("mensajes.csv") as f:
                        mensajes = [line.strip() for line in f]
                    if mensajes:
                        mensaje_a_enviar = random.choice(mensajes)
                        st.success(f"📩 Seleccionado: _{mensaje_a_enviar}_")
                    else:
                        st.error("⚠️ ¡Agrega mensajes primero!")
                except FileNotFoundError:
                    st.error("❌ Archivo no encontrado")
            else:
                mensaje_a_enviar = st.text_area(
                    "Escribe tu mensaje:",
                    height=150,
                    placeholder="¡Feliz cumple {nombre}! 🎉"
                )

            if st.button("🚀 Enviar Correos", key="btn_envio"):
                if not mensaje_a_enviar.strip():
                    st.error("❌ ¡Escribe o selecciona un mensaje!")
                else:
                    success = 0
                    with st.status("Enviando...", expanded=True) as status:
                        for contacto in gestor.contactos:
                            if contacto.correo not in correos_destino:
                                continue
                            prox_cumple = gestor.obtener_proximo_cumpleaños(contacto).replace(year=fecha_prueba.year)
                            if prox_cumple == fecha_prueba:
                                mensaje_final = mensaje_a_enviar.format(nombre=contacto.nombre)
                                try:
                                    if EnviarCorreos(
                                        smtp_server, smtp_port, smtp_user, smtp_pass
                                    ).enviar_correo(
                                        contacto.correo,
                                        "🎉 ¡Feliz Cumple!",
                                        mensaje_final
                                    ):
                                        success += 1
                                        st.write(f"✅ {contacto.nombre}")
                                except Exception as e:
                                    st.error(f"⚠️ No se pudo enviar a {contacto.nombre}: {e}")
                    st.success(f"**Resultado:** ✔️ {success} exitosos | ❌ {len(correos_destino)-success} fallos")

        with msg_col:
            st.subheader("📚 Mensajes Guardados")
            nuevo_msg = st.text_input("Nuevo mensaje:")
            if st.button("💾 Guardar Mensaje"):
                if nuevo_msg.strip():
                    with open("mensajes.csv", "a") as f:
                        f.write(f"{nuevo_msg}\n")
                    st.rerun()

            try:
                with open("mensajes.csv") as f:
                    mensajes = f.readlines()
                if mensajes:
                    for i, msg in enumerate(mensajes):
                        cols = st.columns([6, 1])
                        cols[0].write(f"{i+1}. {msg.strip()}")
                        if cols[1].button("🗑️", key=f"del_{i}"):
                            del mensajes[i]
                            with open("mensajes.csv", "w") as f:
                                f.writelines(mensajes)
                            st.rerun()
            except FileNotFoundError:
                st.warning("📭 No hay mensajes")

        st.markdown("### Correo de prueba")
        dest_prueba = st.text_input("Email de prueba", key="dest_prueba")
        asun_prueba = st.text_input("Asunto de prueba", key="asun_prueba")
        msg_prueba = st.text_area("Mensaje de prueba", key="msg_prueba")
        if st.button("✉️ Enviar correo de prueba", key="btn_prueba"):
            try:
                EnviarCorreos(
                    smtp_server, smtp_port, smtp_user, smtp_pass
                ).enviar_correo(dest_prueba, asun_prueba, msg_prueba)
                st.success("✅ Correo de prueba enviado")
            except Exception as e:
                st.error(f"Error al enviar correo de prueba: {e}")

if __name__ == "__main__":
    from backend_automation import enviar_felicitaciones_automaticas
    threading.Thread(target=enviar_felicitaciones_automaticas, daemon=True).start()
    main()

Overwriting main.py


In [96]:
%%writefile backend_automation.py

import os
import logging
logging.basicConfig(filename="app.log", level=logging.INFO)
from datetime import datetime
import time
import schedule
import random
import csv
from datetime import date
import pytz

from gestor_cumpleaños.almacenamiento import AlmacenamientoCSV
from gestor_cumpleaños.gestor_cumpleaños import GestorCumpleaños
from gestor_cumpleaños.enviar_correos import EnviarCorreos

def obtener_fecha_actual():

     # Configuración dinámica de zona horaria para comparar fechas
     zona_horaria = pytz.timezone("America/Mexico_City")
     return datetime.now(zona_horaria).date()

logging.basicConfig(
    filename="app.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


def enviar_felicitaciones_automaticas():
     try:
         # Validar archivos esenciales
         if not all([os.path.exists("smtp_config.csv"), os.path.exists("mensajes.csv")]):
             print("Error: Archivos de configuración faltantes")
             return

         # Cargar configuración SMTP
         with open("smtp_config.csv", "r") as f:
             server, port, user, pwd = next(csv.reader(f))

         almacenamiento = AlmacenamientoCSV()
         gestor = GestorCumpleaños(almacenamiento)
         enviador = EnviarCorreos(server, int(port), user, pwd)

         # Cargar mensajes
         with open("mensajes.csv", "r") as f:
             mensajes = [line.strip() for line in f]
             if not mensajes:
                 print("Error: No hay mensajes en mensajes.csv")
             if not mensajes:
                print("Error: El archivo mensajes.csv está vacío")
                return

         # Verificar cumpleaños
         hoy = obtener_fecha_actual()
         for contacto in gestor.contactos:
             prox_cumple = gestor.obtener_proximo_cumpleaños(contacto)
             if prox_cumple == hoy:
                 mensaje = random.choice(mensajes).format(nombre=contacto.nombre)
                 if enviador.enviar_correo(contacto.correo, "🎉 ¡Feliz Cumpleaños!", mensaje):
                     logging.info(f"✅ Correo enviado a {contacto.nombre}")
                 else:
                     logging.error(f"🚨 Error crítico: {str(e)}")

     except FileNotFoundError as e:
            logging.critical(f"Archivo faltante: {str(e)}")
     except smtplib.SMTPException as e:
            logging.error(f"Fallo en SMTP: {str(e)}")


# Programar tarea
schedule.every().day.at("08:00").do(enviar_felicitaciones_automaticas)

if __name__ == "__main__":
     while True:
         schedule.run_pending()
         time.sleep(60)

Overwriting backend_automation.py


In [97]:
!mkdir -p gestor_cumpleaños
!mv __init__.py contacto.py gestor_cumpleaños.py enviar_correos.py almacenamiento.py  gestor_cumpleaños/

In [98]:
!ls -l gestor_cumpleaños/

total 24
-rw-r--r-- 1 root root 2006 May 17 23:03 almacenamiento.py
-rw-r--r-- 1 root root  790 May 17 23:03 contacto.py
-rw-r--r-- 1 root root 1806 May 17 23:03 enviar_correos.py
-rw-r--r-- 1 root root 3518 May 17 23:03 gestor_cumpleaños.py
-rw-r--r-- 1 root root  454 May 17 23:03 __init__.py
drwxr-xr-x 2 root root 4096 May 17 22:57 __pycache__


In [99]:
!ls -1 *.py

backend_automation.py
main.py


In [100]:
!rm contactos.csv
!rm config_correo.csv
!rm mensajes.csv
!rm smtp_config.csv

rm: cannot remove 'contactos.csv': No such file or directory
rm: cannot remove 'config_correo.csv': No such file or directory
rm: cannot remove 'mensajes.csv': No such file or directory
rm: cannot remove 'smtp_config.csv': No such file or directory


In [103]:
!streamlit run main.py &> /content/logs.txt & sleep 5 && npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.237.180.91
⠙your url is: https://curly-geckos-happen.loca.lt
